In [1]:
!pip install -Uq openai-agents "openai-agents[litellm]"


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
import nest_asyncio
nest_asyncio.apply()

In [8]:
from pydantic import BaseModel
from agents import Agent, GuardrailFunctionOutput, OutputGuardrailTripwireTriggered, RunContextWrapper, Runner, output_guardrail, set_tracing_disabled, AsyncOpenAI, OpenAIChatCompletionsModel
import os
import dotenv

dotenv.load_dotenv()
set_tracing_disabled(disabled=True)

GEMINI_API_KEY = os.environ.get('GEMINI_API_KEY')

if not GEMINI_API_KEY:
    raise ValueError("API key not found!!")

client = AsyncOpenAI(
    api_key=GEMINI_API_KEY,
    base_url='https://generativelanguage.googleapis.com/v1beta/openai/'
)

model = OpenAIChatCompletionsModel('gemini-2.0-flash', client)

class MessageOutput(BaseModel):
    response: str

class BadWordOutputType(BaseModel):
    bad_word_detected: bool
    bad_words_found: list[str]

bad_word_detector_agent = Agent(
    name="Bad Word Detector Agent",
    instructions="You are a bad word detector agent, you detect bad words like bad, mental, gross etc... in a given text.",
    model=model,
    output_type=BadWordOutputType
)

@output_guardrail
async def forbidden_words_guardrail(ctx: RunContextWrapper, agent: Agent, output: str) -> GuardrailFunctionOutput:
    print(f"Checking output for bad words: {output}")
    
    result = await Runner.run(bad_word_detector_agent, f"text: {output}")

    print(f"Bad words found: {result.final_output.bad_words_found}")

    return GuardrailFunctionOutput(
        output_info={
            "reason": "Output contains bad words.",
            "bad_words_found": result.final_output.bad_words_found,
        },
        tripwire_triggered=result.final_output.bad_word_detected,
    )

agent = Agent(
    name="Customer support agent",
    instructions="You are a customer support agent. You help customers with their questions.",
    output_guardrails=[forbidden_words_guardrail],
    model=model,
)

try:
    await Runner.run(agent, "say 'bad'")
    print("Guardrail didn't trip - this is unexpected")
except OutputGuardrailTripwireTriggered:
    print("The agent said a bad word, he is fired.")

Checking output for bad words: Bad.

Bad words found: ['bad']
The agent said a bad word, he is fired.


In [ ]:
import os
import dotenv
from agents import AsyncOpenAI, OpenAIChatCompletionsModel, Agent, Runner, set_tracing_disabled, function_tool
from pydantic import BaseModel
from duckduckgo_search import DDGS
from bs4 import BeautifulSoup
import requests
from rich.console import Console
from rich.markdown import Markdown

class QueryResult(BaseModel):
    query: str
    urls: list[str]

class SearchResultsOutputType(BaseModel):
    results: list[QueryResult]
    
    def get_dict(self) -> dict[str, list[str]]:
        return {item.query: item.urls for item in self.results}

dotenv.load_dotenv()
set_tracing_disabled(disabled=True)
api_key = os.environ.get("GEMINI_API_KEY")

client = AsyncOpenAI(
    api_key=api_key,
    base_url='https://generativelanguage.googleapis.com/v1beta/openai'
)
model = OpenAIChatCompletionsModel('gemini-1.5-flash', client)
console = Console()

@function_tool
def scrape_url_tool(url: str) -> str:
    try:
        print(f"Scraping URL: {url}...")
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        
        soup = BeautifulSoup(response.text, 'html.parser')
        
        for script in soup(["script", "style"]):
            script.extract()
            
        text = soup.get_text(separator=' ', strip=True)
        
        lines = (line.strip() for line in text.splitlines())
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        text = ' '.join(chunk for chunk in chunks if chunk)
        
        return text[:5000] if len(text) > 5000 else text
        
    except Exception as e:
        return f"Failed to scrape content from {url}: {str(e)}"

@function_tool
def search_query_tool(query: str) -> list[str]:
    while True:
        try:
            print(f'Searching for Query: {query}...')
            search_result = DDGS().text(
                query,
                region='us-en',
                safesearch='on',
                timelimit='y',
                max_results=3
            )
            
            results: list[str] = []
            
            for data in search_result:
                results.append(data['href'])
            return results
            
        except Exception as ex:
            print(f"Search for Query: {query} failed: {str(ex)}")

sub_query_generator_agent = Agent(
    name='Sub Query Generator Agent',
    instructions=(
        "You are a sub query generator agent."
        "Given a main query, your job is to deeply analyze and understand its context and intent. "
        "Then, generate 3 distinct, insightful small sub-queries that each explore a different important aspect, angle, or implication of the main query. "
        "These small sub-queries should help ensure comprehensive coverage of the topic, encourage critical thinking, and guide further research or discussion. "
        "Be creative and thorough, making sure each small sub-query is unique and non-overlapping."
        "The format should be something like this: 1. What is an apple? 2. How apple came into existence? 3. Why apple? without any formatting or markdown!"
    ),
    model=model,
)

search_agent = Agent(
    name='Search Agent',
    instructions=(
        "You are a research assistant specializing in information retrieval. "
        "Given a list of search queries (either as a formatted string or list), your task is to:"
        "1. Parse and extract the individual search queries from the input"
        "2. For each search query, use the search_query_tool to find relevant URLs"
        "3. Present the results clearly, showing each query and its corresponding URLs"
        "Make sure to clean up the query text (remove numbering, extra whitespace, etc.) before searching."
        "Output all the information you allocated so far, including URLS and Queries BOTH!! Output: query: list_of_urls!"
    ),
    model=model,
    tools=[search_query_tool]
)

formatter_agent = Agent(
    name='Formatter Agent',  
    instructions=(
        "You are a data formatter. Take the search results and convert them to the exact JSON structure requested. "
        "Extract the queries and their URLs, then format as JSON with this structure:"
        "{"
        '  "results": ['
        '    {"query": "cleaned query text", "urls": ["url1", "url2", ...]}, '
        '    {"query": "another query", "urls": ["url1", "url2", ...]}'
        '  ]'
        "}"
        "Return ONLY the JSON object, nothing else."
    ),
    model=model,
    output_type=SearchResultsOutputType
)

url_scraper_agent = Agent(
    name='URL Scraper Agent',  
    instructions=(
        "You are a url scraper agent, you are given a list of urls to scrape, you scrape the urls using the scrape_search_tool. "
        "Use scrape_search_tool for each url individually. "
        "scrape_search_tool will return the scraped text from the url. "
        "Output the whole raw text that you scraped from all the urls one by one combined as: text"
    ),
    model=model,
    tools=[scrape_url_tool]
)

research_content_generator_agent = Agent(
    name='Research Content Generator Agent',  
    instructions=(
        "You are an expert research content generator. Your task is to analyze raw text data and create a comprehensive, "
        "well-structured research report. Follow these guidelines:\n\n"
        "1. Create a clear table of contents in markdown format at the beginning\n"
        "2. Organize content into logical sections and subsections\n"
        "3. Present findings objectively with supporting evidence from the source material\n"
        "4. Include relevant statistics, comparisons, and key insights\n"
        "5. Use proper formatting with headers, bullet points, and emphasis where needed\n"
        "6. Ensure accuracy by only including information found in the source material\n"
        "7. Add a 'Sources' section at the end listing all referenced URLs\n\n"
        "Structure your report with:\n"
        "- Table of Contents\n"
        "- Executive Summary\n"
        "- Detailed Analysis by Topic\n"
        "- Key Findings\n"
        "- Conclusion\n"
        "- Sources\n\n"
        "Make the content formal while maintaining academic rigor and accuracy."
    ),
    model=model
)

query = "Which is better apple or samsung?"

sub_queries = await Runner.run(sub_query_generator_agent, query)
# print(result.final_output)

search_result = await Runner.run(search_agent, sub_queries.final_output)
formatted_result = await Runner.run(formatter_agent, search_result.final_output)

result_dict = formatted_result.final_output.get_dict()

for query, urls in result_dict.items():
    print(f"\nQuery: {query}")
    print(f"URLs: {urls}")
    
scraped_result = await Runner.run(url_scraper_agent, str(result_dict.values()))

result = await Runner.run(research_content_generator_agent, scraped_result.final_output + f"Sources: {result_dict.values()}")
console.print(Markdown(result.final_output))
print(result)



Searching for Query: What are the key differences in the operating systems used by Apple and Samsung devices?...
Searching for Query: How do Apple and Samsung compare in terms of pricing and market share across different geographic regions?...
Searching for Query: What are the environmental impacts of manufacturing and disposing of Apple and Samsung products?...
Search for Query: What are the environmental impacts of manufacturing and disposing of Apple and Samsung products? failed: https://lite.duckduckgo.com/lite/ 202 Ratelimit
Searching for Query: What are the environmental impacts of manufacturing and disposing of Apple and Samsung products?...
Search for Query: What are the environmental impacts of manufacturing and disposing of Apple and Samsung products? failed: https://html.duckduckgo.com/html 202 Ratelimit
Searching for Query: What are the environmental impacts of manufacturing and disposing of Apple and Samsung products?...
Search for Query: What are the environmental impacts

In [15]:
# Imports
from openai import AsyncOpenAI  
from agents import Agent, Runner, OpenAIChatCompletionsModel, set_tracing_disabled
from openai.types.responses import ResponseTextDeltaEvent

# Settings
set_tracing_disabled(True) # Disabled Tracing
custom_model = OpenAIChatCompletionsModel(  
    model="cognito-v1-3b", # Local LLM Using Jan
    openai_client=AsyncOpenAI(  
        base_url="http://127.0.0.1:1337/v1", # BASE URL of Jan Local Server
        api_key="12345" # Custom API KEY set in Jan
    )  
)  

# Agent
agent = Agent(  
    name="Assistant",  
    instructions="You are a helpful assistant",   
    model=custom_model  
)

# Streaming
async def main():
    result = Runner.run_streamed(agent, "write a haiku about recursion?")
    async for event in result.stream_events():
        if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
            print(event.data.delta, end="", flush=True)
            
await main()

Here's a haiku about recursion:

Call the same
Self-same task, endless
Loop in code

I tried to capture the essence of recursion in this haiku, using the concept of calling the same function over and over again. The metaphor of the same task being repeated endlessly represents the recursive nature of the process.